In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

In [ ]:
DRIVE_BASE_DIR = '/SMC 09/DDSP' 
INSTRUMENTS_FOLDER = 'instruments'
EXAMPLES_FOLDER = 'examples'

#Install, import and setup files
1.   Install DDSP
2.   Import Python libraries
3.   Copy instruments and examples to Colab

In [ ]:
%tensorflow_version 2.x
!pip install -qU ddsp==1.0.1

In [ ]:
import warnings
import copy
import os
import time
import glob
import gin
import librosa
import pickle
import crepe
import ddsp
import ddsp.training

import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import ipywidgets as widgets

from ddsp.colab import colab_utils
from ddsp.colab.colab_utils import (
    auto_tune, detect_notes, fit_quantile_transform, 
    get_tuning_factor, download, play, record, audio_bytes_to_np,
    specplot, upload, DEFAULT_SAMPLE_RATE)
from google.colab import files
from ipywidgets import interact
from IPython.display import Javascript


sample_rate = DEFAULT_SAMPLE_RATE  # 16000


#Use retina mode
%config InlineBackend.figure_format='retina'

#Ignore warnings
warnings.filterwarnings("ignore")

#Set some globals
REALEXAMPLE = ''
REALINSTRUMENT = ''
audio = None
audio_features = None
audio_features_mod = None
ckpt = None
model = None
DATASET_STATS = None

TRIM = -15

In [ ]:
DRIVE_BASE_DIR = '/content/drive/My Drive' + DRIVE_BASE_DIR
EXAMPLES_DIR = DRIVE_BASE_DIR + '/' + EXAMPLES_FOLDER
assert os.path.exists(EXAMPLES_DIR)
COLAB_EXAMPLES_DIR = 'data/examples' 
!mkdir -p $COLAB_EXAMPLES_DIR

mp3_files = glob.glob(os.path.join(EXAMPLES_DIR, '*.mp3'))
wav_files = glob.glob(os.path.join(EXAMPLES_DIR, '*.wav'))
audio_files = mp3_files + wav_files
example_file_list = [os.path.basename(file) for file in (audio_files)]
example_file_list.sort()

print("Copying {} examples".format(len(example_file_list)))
for fname in audio_files:
    target_name = os.path.join(COLAB_EXAMPLES_DIR, 
                               os.path.basename(fname).replace(' ', '_').replace('\'', '_'))
    #print('  {}'.format(target_name))
    !cp "$fname" $target_name


INSTRUMENTS_DIR = DRIVE_BASE_DIR + '/' + INSTRUMENTS_FOLDER
assert os.path.exists(INSTRUMENTS_DIR)
COLAB_INSTRUMENTS_DIR = 'data/instruments' 
!mkdir -p $COLAB_INSTRUMENTS_DIR

zip_files = glob.glob(os.path.join(INSTRUMENTS_DIR, '*.zip'))
instrument_file_list = [os.path.basename(file) for file in (zip_files)]
instrument_file_list.sort()

print("Copying {} instruments".format(len(instrument_file_list)))
for fname in zip_files:
    target_name = os.path.join(COLAB_INSTRUMENTS_DIR, 
                               os.path.basename(fname).replace(' ', '_').replace('\'', '_'))
    #print('  {}'.format(target_name))
    !cp "$fname" $target_name


# Helper functions

In [ ]:
def load_instrument(instrument_name):

  global REALINSTRUMENT, model, ckpt, DATASET_STATS

  if (REALINSTRUMENT!=instrument_name):
    print('Loading the model...')
    REALINSTRUMENT=instrument_name
    start_time = time.time()
    
    COLAB_MODEL_DIR = COLAB_INSTRUMENTS_DIR + '/current'
    print(f'Unziping {instrument_name}')
    !rm -r $COLAB_MODEL_DIR
    !unzip -j $COLAB_INSTRUMENTS_DIR'/'$instrument_name -d $COLAB_MODEL_DIR &> /dev/null
  
    #gin file
    gin_file = os.path.join(COLAB_MODEL_DIR, 'operative_config-0.gin')
    with gin.unlock_config():
      gin.parse_config_file(gin_file, skip_unknown=True)

    #stats
    dataset_stats_file = os.path.join(COLAB_MODEL_DIR, 'dataset_statistics.pkl')
    print(f'Loading dataset statistics from {dataset_stats_file}')
    try:
      if tf.io.gfile.exists(dataset_stats_file):
        with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
          DATASET_STATS = pickle.load(f)
      else:
          print('WARNING: pickle file not present')
          DATASET_STATS = None
    except Exception as err:
      print('Loading dataset statistics from pickle failed: {}.'.format(err))
  
    #checkpoints
    ckpt_files = [f for f in tf.io.gfile.listdir(COLAB_MODEL_DIR) if 'ckpt' in f]
    ckpt_name = ckpt_files[0].split('.')[0]
    ckpt = os.path.join(COLAB_MODEL_DIR, ckpt_name)
    print('Restoring checkpoint %s took %.1f seconds' % (ckpt_name, time.time() - start_time))

    if REALEXAMPLE != '' and REALINSTRUMENT != '':
      print("Restoring model from load_instrument")
      restore_model()

In [ ]:
def load_example(example_name):

  global REALEXAMPLE, audio, audio_features

  if (REALEXAMPLE!=example_name):
    print('Loading audio and extracting features...')
    REALEXAMPLE=example_name
    start_time = time.time()

    with open(COLAB_EXAMPLES_DIR+'/'+REALEXAMPLE, 'rb') as fd:
      contents = fd.read()

    audio = audio_bytes_to_np(contents, sample_rate=DEFAULT_SAMPLE_RATE, normalize_db=None)
    audio = audio[np.newaxis, :]

    #Compute features.
    ddsp.spectral_ops.reset_crepe()
    audio_features = ddsp.training.metrics.compute_audio_features(audio)
    audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
    print('Operation took %.1f seconds' % (time.time() - start_time))

    if REALEXAMPLE != '' and REALINSTRUMENT != '':
      print("Restoring model from load_example")
      restore_model()


In [ ]:
def restore_model():
  global model, audio_features, ckpt

  #Ensure dimensions and sampling rates are equal
  time_steps_train = gin.query_parameter('F0LoudnessPreprocessor.time_steps')
  n_samples_train = gin.query_parameter('Harmonic.n_samples')
  hop_size = int(n_samples_train / time_steps_train)

  time_steps = int(audio.shape[1] / hop_size)
  n_samples = time_steps * hop_size

  gin_params = [
      'Harmonic.n_samples = {}'.format(n_samples),
      'FilteredNoise.n_samples = {}'.format(n_samples),
      'F0LoudnessPreprocessor.time_steps = {}'.format(time_steps),
      'oscillator_bank.use_angular_cumsum = True',  # Avoids cumsum accumulation errors.
  ]

  with gin.unlock_config():
    gin.parse_config(gin_params)

  # Trim all input vectors to correct lengths 
  for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
      audio_features[key] = audio_features[key][:time_steps]
  audio_features['audio'] = audio_features['audio'][:, :n_samples]

  #Set up the model just to predict audio given new conditioning
  model = ddsp.training.models.Autoencoder()
  model.restore(ckpt)

In [ ]:
def shift_loudness(audio_features, ld_shift=0.0):
  #Shift loudness by a number of dB
  audio_features['loudness_db'] += ld_shift
  return audio_features


def shift_f0(audio_features, octave_shift=0.0):
  #Shift f0 by a number of octaves
  audio_features['f0_hz'] *= 2.0 ** (octave_shift)
  audio_features['f0_hz'] = np.clip(audio_features['f0_hz'], 
                                    0.0, 
                                    librosa.midi_to_hz(110.0))
  return audio_features

In [ ]:
def compute_audio_features():

  global audio_features_mod, mask_on, loudness_norm, note_on_ratio, f0_midi_at

  audio_features_mod = {k: v.copy() for k, v in audio_features.items()}

  mask_on, note_on_ratio = detect_notes(audio_features['loudness_db'],
                                        audio_features['f0_confidence'],
                                        mask_threshold, 
                                        exponent=2.0)

  if DATASET_STATS is not None:
    _, loudness_norm = fit_quantile_transform(audio_features['loudness_db'],
                                              mask_on,
                                              inv_quantile=DATASET_STATS['quantile_transform'])

    mask_off = np.logical_not(mask_on)
    loudness_norm[mask_off] -=  quiet * (1.0 - note_on_ratio[mask_off][:, np.newaxis])
    loudness_norm = np.reshape(loudness_norm, audio_features['loudness_db'].shape) 
    if use_norm:
      audio_features_mod['loudness_db'] = loudness_norm 


  if autotune_amount > 0:
    f0_midi = np.array(ddsp.core.hz_to_midi(audio_features_mod['f0_hz']))
    tuning_factor = get_tuning_factor(f0_midi, audio_features_mod['f0_confidence'], mask_on)
    f0_midi_at = auto_tune(f0_midi, tuning_factor, mask_on, amount=autotune_amount, chromatic=True)
    audio_features_mod['f0_hz'] = ddsp.core.midi_to_hz(f0_midi_at)

  audio_features_mod = shift_loudness(audio_features_mod, loudness_shift)
  audio_features_mod = shift_f0(audio_features_mod, octave_shift)

In [ ]:
def run_model():

  # Run a batch of predictions.
  start_time = time.time()

  outputs = model(audio_features_mod, training=False)
  audio_gen = model.get_audio_from_outputs(outputs)

  #Old code for DDSP v0.13.0
  #controls =  model.get_controls(audio_features_mod, training=False)
  #audio_gen = controls['processor_group']['signal']

  #with out:
  print('Prediction took %.1f seconds' % (time.time() - start_time))

  # Plot
  if full_output_checkbox.value:
    print('Original')
    play(audio)

  print('Resynthesis')
  play(audio_gen)

  if full_output_checkbox.value:
    specplot(audio)
    plt.title("Original")

  specplot(audio_gen)
  _ = plt.title("Resynthesis")

In [ ]:
def plot_data():
  #auto octave adjustment.
  pitch = ddsp.core.hz_to_midi(audio_features['f0_hz'])
  example_mean_pitch = np.mean(pitch[mask_on])
  if DATASET_STATS is not None:
    instrument_mean_pitch = DATASET_STATS['mean_pitch']
  else:
    instrument_mean_pitch = example_mean_pitch
  #p_diff = instrument_mean_pitch - example_mean_pitch
  #p_diff_octave = p_diff / 12.0
  #round_fn = np.floor if p_diff_octave > 1.5 else np.ceil
  #p_diff_octave = round_fn(p_diff_octave)
  #audio_features_mod = shift_f0(audio_features_mod, p_diff_octave)

  #with out:
  fig, ax = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(15.5, 10))
  ax[0].plot(audio_features['loudness_db'])
  if DATASET_STATS is not None:
    ax[0].plot(loudness_norm)
    ax[0].legend(['Original', 'Norm'])
  else:
    ax[0].legend(['Original'])
  ax[0].set_ylabel('Loudness')
  ax[0].set_ylim(-100,0)
  ax[1].plot(librosa.hz_to_midi(audio_features['f0_hz']))
  ax[1].plot(np.ones_like(mask_on) * instrument_mean_pitch, ':')
  ax[1].plot(np.ones_like(mask_on) * example_mean_pitch, ':')
  if autotune_amount > 0:
    ax[1].plot(f0_midi_at)
    ax[1].legend(['Pitch', 'Instrument pitch','Example pitch', 'autotune'])
  else:
    ax[1].legend(['Pitch', 'Instrument pitch','Example pitch'])
  ax[1].set_ylabel('f0 [MIDI]')
  ax[1].set_ylim(bottom=36)
  ax[2].plot(audio_features['f0_confidence'])
  ax[2].set_ylabel('f0 confidence')
  ax[3].plot(note_on_ratio)
  ax[3].plot(np.ones_like(mask_on) * mask_threshold, 'k:')
  ax[3].plot(mask_on)
  ax[3].set_ylabel('Note-on Mask')
  ax[3].legend(['Note-on ratio', 'Threshold', 'Mask'])
  _=ax[3].set_xlabel('Time step [frame]')

#GUI components

In [ ]:
instrument_widget = widgets.Dropdown(options = instrument_file_list, value=instrument_file_list[0], description='Instrument:')
example_widget = widgets.Dropdown(options = example_file_list, value=example_file_list[0], description='Example:')

go_button = widgets.Button(
    description='Transfer timbre',
    button_style='primary', 
    layout=widgets.Layout(width='auto')
)

def go_eventhandler(change):
  
  global use_norm, mask_threshold, quiet, autotune_amount, octave_shift, loudness_shift

  #out.clear_output()

  use_norm = use_norm_checkbox.value
  mask_threshold = mask_threshold_slider.value
  quiet = quiet_slider.value
  autotune_amount = autotune_amount_slider.value
  octave_shift = octave_shift_slider.value
  loudness_shift = loudness_shift_slider.value

  #with out:
  print()
  print("EXAMPLE = '%s'" % REALEXAMPLE)
  print("INSTRUMENT = '%s" % REALINSTRUMENT)
  print('use_norm = %s' % ('True' if use_norm else 'False'))
  print('mask_threshold = %.2f' % mask_threshold)
  print('quiet = %d' % quiet)
  print('autotune_amount = %.2f' % autotune_amount)
  print('octave_shift = %.2f' % octave_shift)
  print('loudness_shift = %d' % loudness_shift)
  print()

  compute_audio_features()
  run_model()
  if full_output_checkbox.value:
    plot_data()

go_button.on_click(go_eventhandler)

full_output_checkbox = widgets.Checkbox(
    value=False,
    description='Show full output'
)

use_norm_checkbox = widgets.Checkbox(
    value=False,
    description='Use loudness statistics'
)

mask_threshold_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=2,
    step=0.1,
    description='Mask threshold:',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

quiet_slider = widgets.IntSlider(
    value=20,
    min=0,
    max=50,
    step=1,
    description='Attenuation:',
    continuous_update=False,
    readout=True,
    readout_format='d'
)
autotune_amount_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='Autotune:',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)
octave_shift_slider = widgets.IntSlider(
    value=0,
    min=-3,
    max=3,
    step=1,
    description='Octave shift:',
    continuous_update=False,
    readout=True,
    readout_format='d'
)

loudness_shift_slider = widgets.IntSlider(
    value=0,
    min=-30,
    max=30,
    step=5,
    description='Loudness shift:',
    continuous_update=False,
    readout=True,
    readout_format='d'
)

#Set instrument and example

In [ ]:
interact(load_instrument, instrument_name=instrument_widget)
interact(load_example, example_name=example_widget)

#Preprocessing

In [ ]:
#Improve loudness with statistical info
display(full_output_checkbox)
display(use_norm_checkbox)
display(mask_threshold_slider)
display(quiet_slider)

#Improve pitch
display(autotune_amount_slider)

#Adjust source material to model pitch and loudness
display(octave_shift_slider)
display(loudness_shift_slider)

display(go_button)
